In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# remover comentarios si sed quiere instalar la libreria 
# publicada en pipy
#%pip uninstall azure-datalake-utils -y
#%pip install -i https://test.pypi.org/simple/ --extra-index-url=https://pypi.org/simple azure-datalake-utils==0.2.0.dev23
#%pip show azure-datalake-utils

In [3]:
import logging
import sys
sys.path.append('../')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

Asegurar crear primero el archivo `.env` con las siguientes variables de ambiente:
```
datalake="nombre_datalake_cuenta"
datalake_key="xxxkey_datalake_cuenta_zxccxxcx"
AZURE_STORAGE_CONNECTION_STRING="aaaa"
CONTENEDOR_VALIDACIONES="calidad-datos" # debe existir en la cuenta.
````

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from centraal_dataframework.resources import datalake
info = datalake.read_csv("test/test_framework.csv", sep="|", decimal = ",").drop(columns=['Unnamed: 0'])

In [6]:
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from centraal_dataframework.tasks import task_dq

@task_dq
def hello_gx(datalake, gx_toolkit, logger):
    """Saluda al gx."""
    source = datalake.read_csv("test/test_framework.csv", sep="|")
    logger.info(source.head(1))
    logger.info("hello there, iam a task_dq using great expectations")
    # creaciones de expectativas
    expectation_1 = ExpectationConfiguration(
        expectation_type="expect_table_columns_to_match_set",
        kwargs={'column_set': ['Producto', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01',
        '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01',
        '2024-04-01', '2024-05-01', '2024-06-01']},
        meta={
            "notes": {
                "format": "markdown",
                "content": "Ejemplo de expectativa",
            }
        },
    )
    expectation_2 = ExpectationConfiguration(
        expectation_type = "expect_column_values_to_not_be_null",
        kwargs = {"column" : "Producto"} 
    )

    expectation_3 = ExpectationConfiguration(
        expectation_type = "expect_table_row_count_to_be_between",
        kwargs = {"max_value" : 160, "min_value": 131},
        meta={
            "notes": {
                "format": "markdown",
                "content": "**Ejemplo de expectativa** que va fallar",
            }
        }
    )
    url = gx_toolkit.run_expectations_on_df(source, "test", [expectation_1, expectation_2, expectation_3])
    print("reporte de expectativas", url)

In [8]:
hello_gx()

TAREA: hello_gx--2023-10-23 10:57:01,532-INFO-   Unnamed: 0  Producto 2023-07-01 2023-08-01 2023-09-01 2023-10-01  \
0           0   9002263      237,0      700,0     3200,0     3200,0   

  2023-11-01 2023-12-01 2024-01-01 2024-02-01 2024-03-01 2024-04-01  \
0     3200,0     2920,0     2920,0      398,0      336,0      295,0   

  2024-05-01 2024-06-01  
0      336,0      224,0  
TAREA: hello_gx--2023-10-23 10:57:01,532-INFO-   Unnamed: 0  Producto 2023-07-01 2023-08-01 2023-09-01 2023-10-01  \
0           0   9002263      237,0      700,0     3200,0     3200,0   

  2023-11-01 2023-12-01 2024-01-01 2024-02-01 2024-03-01 2024-04-01  \
0     3200,0     2920,0     2920,0      398,0      336,0      295,0   

  2024-05-01 2024-06-01  
0      336,0      224,0  
TAREA: hello_gx--2023-10-23 10:57:01,549-INFO-hello there, iam a task_dq using great expectations
TAREA: hello_gx--2023-10-23 10:57:01,549-INFO-hello there, iam a task_dq using great expectations


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

reporte de expectativas https://centraaldl.z13.web.core.windows.net/validations/hello_gx_expectation_suite/__none__/20231023T155701.835505Z/hello_gx_source-hello_gx_source_test.html


Crear el modulo con la logica, en la siguiente celda se configura una logica de pandas.

In [5]:
%%writefile logica_pandas.py
"""Prueba."""
from centraal_dataframework.tasks import task


@task
def hello_datalake(datalake, logger):
    """Saluda al datalake."""
    source = datalake.read_csv("test/test_framework.csv", sep="|")
    logger.info(source.head(1))
    logger.info("hello there, iam a task")


Overwriting logica_pandas.py


O puedes crear una estructura mas compleja de modulos. en la siguiente celda se crea un modulo dentro de otra carpeta y se usa un decorador para poder usar great expectations.

In [6]:
%mkdir modulo

mkdir: cannot create directory ‘modulo’: File exists


In [7]:
%%writefile modulo/__init__.py
"""Prueba."""

Overwriting modulo/__init__.py


In [8]:
%%writefile modulo/logica_gx.py
"""Prueba gx."""
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from centraal_dataframework.tasks import task_dq


@task_dq
def hello_gx(datalake, gx_toolkit, logger):
    """Saluda al gx."""
    source = datalake.read_csv("test/test_framework.csv", sep="|")
    logger.info(source.head(1))
    logger.info("hello there, iam a task_dq using great expectations")
    # creaciones de expectativas
    expectation_1 = ExpectationConfiguration(
        expectation_type="expect_table_columns_to_match_set",
        kwargs={'column_set': ['Producto', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01',
        '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01',
        '2024-04-01', '2024-05-01', '2024-06-01']},
        meta={
            "notes": {
                "format": "markdown",
                "content": "Ejemplo de expectativa",
            }
        },
    )
    expectation_2 = ExpectationConfiguration(
        expectation_type = "expect_column_values_to_not_be_null",
        kwargs = {"column" : "Producto"} 
    )

    expectation_3 = ExpectationConfiguration(
        expectation_type = "expect_table_row_count_to_be_between",
        kwargs = {"max_value" : 160, "min_value": 131},
        meta={
            "notes": {
                "format": "markdown",
                "content": "**Ejemplo de expectativa** que va fallar",
            }
        }
    )
    url = gx_toolkit.run_expectations_on_df(source, "test", [expectation_1, expectation_2, expectation_3])
    print("reporte de expectativas", url)


Overwriting modulo/logica_gx.py


A continuación un se muestra el contenido que debe tener `function_app.py`

In [9]:
# contenido de function_app.py
import azure.functions as func
from centraal_dataframework.blueprints import framework
# se deben importar las funciones, ejemplo todas las funciones dentro
# de uno
from logica_pandas import *
# o se pueden importar uno a uno
from modulo.logica_gx import hello_gx
from modulo.logica_gx import hello_gx_negocio

app = func.FunctionApp()
# se agrega el framework:
app.register_functions(framework)

Realizar test dentro del notebook.

In [10]:
from azure.functions import QueueMessage

queue_trigger = app.get_functions()[1].get_user_function()
# simular la ejecucion de la tarea.
queue_trigger(QueueMessage(body = bytes("otra_cosa",  "utf-8")))

se presento error en otra_cosa
Traceback (most recent call last):
  File "/home/german/repos/centraal-dataframework/centraal_dataframework/blueprints.py", line 62, in execute_tasks_inqueue
    func_to_execute()
TypeError: 'NoneType' object is not callable


ErrorEnTarea: Se encontro un error en otra_cosa.

In [ ]:
# simular la ejecucion de la tarea.
queue_trigger(QueueMessage(body = bytes("otra_cosa",  "utf-8")))